# Resample Audio — Folder Batch

Resamples all WAV files to a **target sample rate** and forces **mono**.
Useful for standardizing datasets before training (e.g. 22050 Hz for LJSpeech, 16000 Hz for STT).

---

> No GPU required. Uses `librosa` + `soundfile`.

In [ ]:
import os

# ════════════════════════════════════════════
# ⚙️  CONFIGURATION
# ════════════════════════════════════════════
os.environ.setdefault("INPUT_DIR",   "/data/audio_in")
os.environ.setdefault("OUTPUT_DIR",  "/data/audio_resampled")
os.environ.setdefault("TARGET_SR",   "22050")     # Target sample rate in Hz
os.environ.setdefault("MONO",        "1")          # 1 = force mono

INPUT_DIR  = os.environ["INPUT_DIR"]
OUTPUT_DIR = os.environ["OUTPUT_DIR"]
TARGET_SR  = int(os.environ["TARGET_SR"])
MONO       = os.environ.get("MONO", "1").strip() in ("1", "true", "yes")

print(f"Input:     {INPUT_DIR}")
print(f"Output:    {OUTPUT_DIR}")
print(f"Target SR: {TARGET_SR} Hz")
print(f"Mono:      {MONO}")

In [ ]:
!pip install --quiet --break-system-packages librosa soundfile tqdm

In [ ]:
import os
import librosa
import soundfile as sf
from pathlib import Path
from tqdm import tqdm

os.makedirs(OUTPUT_DIR, exist_ok=True)

files = sorted(Path(INPUT_DIR).rglob("*.wav"))
print(f"Found {len(files)} WAV files")

done, skipped, failed = 0, 0, 0
for src in tqdm(files, desc="Resampling", unit="file"):
    rel = src.relative_to(INPUT_DIR)
    dst = Path(OUTPUT_DIR) / rel
    dst.parent.mkdir(parents=True, exist_ok=True)

    if dst.exists():
        skipped += 1
        continue

    try:
        y, sr = librosa.load(str(src), sr=TARGET_SR, mono=MONO)
        sf.write(str(dst), y, TARGET_SR)
        done += 1
    except Exception as e:
        tqdm.write(f"  FAIL {src.name}: {e}")
        failed += 1

print(f"\nDone: {done} resampled, {skipped} skipped, {failed} failed")